# Export Master File

This script:

* unzips .zip files;
* reads them as separate data frames;
* binds them together into one data frame; and
* exports the results as a .csv file

In [11]:
# load necessary packages
import pandas
import zipfile
import os
import re

In [12]:
# set working directory
directory = "/Users/cristiannuno/Desktop/Advanced_Analytics/DABP_Group_Project/raw_data/"
os.chdir(directory)

In [29]:
# store all files in current wd
file_names = os.listdir(".")

# create regex to only give back file names that contain .zip
my_pattern = re.compile(".zip$")

# store zip files that satisfy the regex
my_zip_files = list(filter(my_pattern.search, file_names))


['2018_fsa_acres_sep0618_csa1983.zip', '2016_fsa_acres_jan2017_edr32.zip', '2008_fsa_acres_sum_final.zip', '2007_fsa_acres_sum_final.zip', '2013_fsa_acres_jan_2014.zip', '2017_fsa_acres_jan2018.zip', '2015_fsa_acres_Jan2016_sq19.zip', '2009_fsa_acres_sum_final_8.zip', '2014_fsa_acres_Jan2014.zip', '2010_fsa_acres_sum_final_6.zip', '2012_fsa_acres_jan_2013.zip', '2011_fsa_acres_sum_jan2012.zip']


In [34]:
# unzip each file in my_zip_files
for i in my_zip_files:
    print(i)
    zip = zipfile.ZipFile(file = i, mode = "r")
    zip.extractall(os.getcwd())
    zip.close()

2018_fsa_acres_sep0618_csa1983.zip
2016_fsa_acres_jan2017_edr32.zip
2008_fsa_acres_sum_final.zip
2007_fsa_acres_sum_final.zip
2013_fsa_acres_jan_2014.zip
2017_fsa_acres_jan2018.zip
2015_fsa_acres_Jan2016_sq19.zip
2009_fsa_acres_sum_final_8.zip
2014_fsa_acres_Jan2014.zip
2010_fsa_acres_sum_final_6.zip
2012_fsa_acres_jan_2013.zip
2011_fsa_acres_sum_jan2012.zip


In [33]:
os.getcwd()

'/Users/cristiannuno/Desktop/Advanced_Analytics/DABP_Group_Project/raw_data'